In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
sns.set()

In [376]:
# Load the dataset
df = pd.read_csv('../data/SteamCharts.csv', encoding='unicode_escape')
df_st = pd.read_csv('../data/games.csv')

In [340]:
df.head()

,gamename,year,month,avg,gain,peak,avg_peak_perc
0,Counter-Strike: Global Offensive,2021,February,741013.24,-2196.42,1123485,65.9567%
1,Counter-Strike: Global Offensive,2021,January,743209.66,25405.91,1124553,66.0893%
2,Counter-Strike: Global Offensive,2020,December,717803.75,49049.17,1164396,61.646%
3,Counter-Strike: Global Offensive,2020,November,668754.58,55087.89,1037464,64.4605%
4,Counter-Strike: Global Offensive,2020,October,613666.69,6816.37,943876,65.0156%


In [341]:
df_st.head()

,AppID,Name,Release date,Estimated owners,Peak CCU,Required age,Price,DLC count,About the game,Supported languages,...,Average playtime two weeks,Median playtime forever,Median playtime two weeks,Developers,Publishers,Categories,Genres,Tags,Screenshots,Movies
0,20200,Galactic Bowling,"Oct 21, 2008",0 - 20000,0,0,19.99,0,Galactic Bowling is an exaggerated and stylize...,['English'],...,0,0,0,Perpetual FX Creative,Perpetual FX Creative,"Single-player,Multi-player,Steam Achievements,...","Casual,Indie,Sports","Indie,Casual,Sports,Bowling",https://cdn.akamai.steamstatic.com/steam/apps/...,http://cdn.akamai.steamstatic.com/steam/apps/2...
1,655370,Train Bandit,"Oct 12, 2017",0 - 20000,0,0,0.99,0,THE LAW!! Looks to be a showdown atop a train....,"['English', 'French', 'Italian', 'German', 'Sp...",...,0,0,0,Rusty Moyher,Wild Rooster,"Single-player,Steam Achievements,Full controll...","Action,Indie","Indie,Action,Pixel Graphics,2D,Retro,Arcade,Sc...",https://cdn.akamai.steamstatic.com/steam/apps/...,http://cdn.akamai.steamstatic.com/steam/apps/2...
2,1732930,Jolt Project,"Nov 17, 2021",0 - 20000,0,0,4.99,0,Jolt Project: The army now has a new robotics ...,"['English', 'Portuguese - Brazil']",...,0,0,0,Campião Games,Campião Games,Single-player,"Action,Adventure,Indie,Strategy",NaN,https://cdn.akamai.steamstatic.com/steam/apps/...,http://cdn.akamai.steamstatic.com/steam/apps/2...
3,1355720,Henosis™,"Jul 23, 2020",0 - 20000,0,0,5.99,0,HENOSIS™ is a mysterious 2D Platform Puzzler w...,"['English', 'French', 'Italian', 'German', 'Sp...",...,0,0,0,Odd Critter Games,Odd Critter Games,"Single-player,Full controller support","Adventure,Casual,Indie","2D Platformer,Atmospheric,Surreal,Mystery,Puzz...",https://cdn.akamai.steamstatic.com/steam/apps/...,http://cdn.akamai.steamstatic.com/steam/apps/2...
4,1139950,Two Weeks in Painland,"Feb 3, 2020",0 - 20000,0,0,0.00,0,ABOUT THE GAME Play as a hacker who has arrang...,"['English', 'Spanish - Spain']",...,0,0,0,Unusual Games,Unusual Games,"Single-player,Steam Achievements","Adventure,Indie","Indie,Adventure,Nudity,Violent,Sexual Content,...",https://cdn.akamai.steamstatic.com/steam/apps/...,http://cdn.akamai.steamstatic.com/steam/apps/2...


In [378]:
# Popularity - Clean and format the data
df['year'] = df['year'].astype(str).str.strip()
df['month'] = df['month'].astype(str).str.strip()
df['date'] = pd.to_datetime(df['year'] + " " + df['month'], format="%Y %B", errors='coerce')
df.columns = df.columns.str.replace(' ', '_')
df.columns = df.columns.str.lower()
print(df.shape)
df.head()

(83789, 8)


,gamename,year,month,avg,gain,peak,avg_peak_perc,date
0,Counter-Strike: Global Offensive,2021,February,741013.24,-2196.42,1123485,65.9567%,2021-02-01
1,Counter-Strike: Global Offensive,2021,January,743209.66,25405.91,1124553,66.0893%,2021-01-01
2,Counter-Strike: Global Offensive,2020,December,717803.75,49049.17,1164396,61.646%,2020-12-01
3,Counter-Strike: Global Offensive,2020,November,668754.58,55087.89,1037464,64.4605%,2020-11-01
4,Counter-Strike: Global Offensive,2020,October,613666.69,6816.37,943876,65.0156%,2020-10-01


In [343]:
df_st.isna().sum()

AppID                             0
Name                              2
Release date                      0
Estimated owners                  0
Peak CCU                          0
Required age                      0
Price                             0
DLC count                         0
About the game                 2917
Supported languages               0
Full audio languages              0
Reviews                       67625
Header image                      0
Website                       40308
Support url                   38759
Support email                 12090
Windows                           0
Mac                               0
Linux                             0
Metacritic score                  0
Metacritic url                73195
User score                        0
Positive                          0
Negative                          0
Score rank                    76945
Achievements                      0
Recommendations                   0
Notes                       

In [379]:
# Steam games - Filling up Null values
df_st = df_st.dropna(subset = ['Name'])
values = {
    "About the game": "N/A", 
#     "Full audio languages": ["N/A"], 
    "Reviews": "N/A", 
    "Website": "N/A", 
    "Support url": "N/A", 
    "Support email": "N/A", 
    "Metacritic url": "N/A", 
    "Notes": "N/A", 
    "Developers": "N/A", 
    "Publishers": "N/A", 
    "Categories": "Undefined", 
    "Genres": "Undefined", 
    "Tags": "N/A", 
    "Screenshots": "N/A", 
    "Movies": "N/A", 
}
df_st = df_st.fillna(value=values)
df_st.head()

,AppID,Name,Release date,Estimated owners,Peak CCU,Required age,Price,DLC count,About the game,Supported languages,...,Average playtime two weeks,Median playtime forever,Median playtime two weeks,Developers,Publishers,Categories,Genres,Tags,Screenshots,Movies
0,20200,Galactic Bowling,"Oct 21, 2008",0 - 20000,0,0,19.99,0,Galactic Bowling is an exaggerated and stylize...,['English'],...,0,0,0,Perpetual FX Creative,Perpetual FX Creative,"Single-player,Multi-player,Steam Achievements,...","Casual,Indie,Sports","Indie,Casual,Sports,Bowling",https://cdn.akamai.steamstatic.com/steam/apps/...,http://cdn.akamai.steamstatic.com/steam/apps/2...
1,655370,Train Bandit,"Oct 12, 2017",0 - 20000,0,0,0.99,0,THE LAW!! Looks to be a showdown atop a train....,"['English', 'French', 'Italian', 'German', 'Sp...",...,0,0,0,Rusty Moyher,Wild Rooster,"Single-player,Steam Achievements,Full controll...","Action,Indie","Indie,Action,Pixel Graphics,2D,Retro,Arcade,Sc...",https://cdn.akamai.steamstatic.com/steam/apps/...,http://cdn.akamai.steamstatic.com/steam/apps/2...
2,1732930,Jolt Project,"Nov 17, 2021",0 - 20000,0,0,4.99,0,Jolt Project: The army now has a new robotics ...,"['English', 'Portuguese - Brazil']",...,0,0,0,Campião Games,Campião Games,Single-player,"Action,Adventure,Indie,Strategy",N/A,https://cdn.akamai.steamstatic.com/steam/apps/...,http://cdn.akamai.steamstatic.com/steam/apps/2...
3,1355720,Henosis™,"Jul 23, 2020",0 - 20000,0,0,5.99,0,HENOSIS™ is a mysterious 2D Platform Puzzler w...,"['English', 'French', 'Italian', 'German', 'Sp...",...,0,0,0,Odd Critter Games,Odd Critter Games,"Single-player,Full controller support","Adventure,Casual,Indie","2D Platformer,Atmospheric,Surreal,Mystery,Puzz...",https://cdn.akamai.steamstatic.com/steam/apps/...,http://cdn.akamai.steamstatic.com/steam/apps/2...
4,1139950,Two Weeks in Painland,"Feb 3, 2020",0 - 20000,0,0,0.00,0,ABOUT THE GAME Play as a hacker who has arrang...,"['English', 'Spanish - Spain']",...,0,0,0,Unusual Games,Unusual Games,"Single-player,Steam Achievements","Adventure,Indie","Indie,Adventure,Nudity,Violent,Sexual Content,...",https://cdn.akamai.steamstatic.com/steam/apps/...,http://cdn.akamai.steamstatic.com/steam/apps/2...


In [380]:
# Steam games - Clean and format the data
df_st.columns = df_st.columns.str.replace(' ', '_')
df_st.columns = df_st.columns.str.lower()
df_st = df_st.rename(columns = {"name": "gamename"})
df_st['release_date'] = pd.to_datetime(df_st['release_date'])
df_st['release_year'] = df_st['release_date'].dt.strftime('%Y')
df_st.head()

,appid,gamename,release_date,estimated_owners,peak_ccu,required_age,price,dlc_count,about_the_game,supported_languages,...,median_playtime_forever,median_playtime_two_weeks,developers,publishers,categories,genres,tags,screenshots,movies,release_year
0,20200,Galactic Bowling,2008-10-21,0 - 20000,0,0,19.99,0,Galactic Bowling is an exaggerated and stylize...,['English'],...,0,0,Perpetual FX Creative,Perpetual FX Creative,"Single-player,Multi-player,Steam Achievements,...","Casual,Indie,Sports","Indie,Casual,Sports,Bowling",https://cdn.akamai.steamstatic.com/steam/apps/...,http://cdn.akamai.steamstatic.com/steam/apps/2...,2008
1,655370,Train Bandit,2017-10-12,0 - 20000,0,0,0.99,0,THE LAW!! Looks to be a showdown atop a train....,"['English', 'French', 'Italian', 'German', 'Sp...",...,0,0,Rusty Moyher,Wild Rooster,"Single-player,Steam Achievements,Full controll...","Action,Indie","Indie,Action,Pixel Graphics,2D,Retro,Arcade,Sc...",https://cdn.akamai.steamstatic.com/steam/apps/...,http://cdn.akamai.steamstatic.com/steam/apps/2...,2017
2,1732930,Jolt Project,2021-11-17,0 - 20000,0,0,4.99,0,Jolt Project: The army now has a new robotics ...,"['English', 'Portuguese - Brazil']",...,0,0,Campião Games,Campião Games,Single-player,"Action,Adventure,Indie,Strategy",N/A,https://cdn.akamai.steamstatic.com/steam/apps/...,http://cdn.akamai.steamstatic.com/steam/apps/2...,2021
3,1355720,Henosis™,2020-07-23,0 - 20000,0,0,5.99,0,HENOSIS™ is a mysterious 2D Platform Puzzler w...,"['English', 'French', 'Italian', 'German', 'Sp...",...,0,0,Odd Critter Games,Odd Critter Games,"Single-player,Full controller support","Adventure,Casual,Indie","2D Platformer,Atmospheric,Surreal,Mystery,Puzz...",https://cdn.akamai.steamstatic.com/steam/apps/...,http://cdn.akamai.steamstatic.com/steam/apps/2...,2020
4,1139950,Two Weeks in Painland,2020-02-03,0 - 20000,0,0,0.00,0,ABOUT THE GAME Play as a hacker who has arrang...,"['English', 'Spanish - Spain']",...,0,0,Unusual Games,Unusual Games,"Single-player,Steam Achievements","Adventure,Indie","Indie,Adventure,Nudity,Violent,Sexual Content,...",https://cdn.akamai.steamstatic.com/steam/apps/...,http://cdn.akamai.steamstatic.com/steam/apps/2...,2020


In [381]:
# Adding is multiplayer
# df_st['multi-player']*=1

# Category - multi/single player
df_st['multi_player'] = (df_st.categories.str.contains('Multi-player')==True)*1
df_st['pvp'] = (df_st.categories.str.contains('PvP')==True)*1
df_st['co-op'] = (df_st.categories.str.contains('Co-op')==True)*1

# Genres
df_st['genre_action'] = (df_st.genres.str.contains('Action')==True)*1
df_st['genre_adventure'] = (df_st.genres.str.contains('Adventure')==True)*1
df_st['genre_casual'] = (df_st.genres.str.contains('Casual')==True)*1
df_st['genre_sexual_content'] = (df_st.genres.str.contains('Sexual Content')==True)*1
df_st['genre_strategy'] = (df_st.genres.str.contains('Strategy')==True)*1
df_st['genre_sports'] = (df_st.genres.str.contains('Sports')==True)*1
df_st['genre_racing'] = (df_st.genres.str.contains('Racing')==True)*1
df_st['genre_rpg'] = (df_st.genres.str.contains('RPG')==True)*1
df_st['genre_simulation'] = (df_st.genres.str.contains('Simulation')==True)*1
df_st['indie'] = (df_st.genres.str.contains('Indie')==True)*1

# Setting option
df_st['full_audio'] = (df_st['full_audio_languages'] != '[]')*1
df_st['full_controller_support'] = (df_st.categories.str.contains('Full controller support')==True)*1
df_st['windows'] = df_st.windows*1
df_st['linux'] = df_st.linux*1
df_st['mac'] = df_st.mac*1

In [382]:
# Entertainment Software Rating Board (ESRB) ['0+', '10+', '13+', '18+']
print(df_st.required_age.unique())
lower = 0; upper = 13
condition = (df_st['required_age']>=lower) & (df_st['required_age']<upper)

df_st['age_0_plus'] = condition*1
lower = 13; upper = 18
condition = (df_st['required_age']>=lower) & (df_st['required_age']<upper)
df_st['age_13_plus'] = condition*1
lower = 18; upper = 999
condition = (df_st['required_age']>=lower) & (df_st['required_age']<upper)
df_st['age_18_plus'] = condition*1

df_st.head()

[ 0 17 18 10 13 16  6 14 12 15  3 21  7  5 20  1 19  9]


,appid,gamename,release_date,estimated_owners,peak_ccu,required_age,price,dlc_count,about_the_game,supported_languages,...,genre_sports,genre_racing,genre_rpg,genre_simulation,indie,full_audio,full_controller_support,age_0_plus,age_13_plus,age_18_plus
0,20200,Galactic Bowling,2008-10-21,0 - 20000,0,0,19.99,0,Galactic Bowling is an exaggerated and stylize...,['English'],...,1,0,0,0,1,0,0,1,0,0
1,655370,Train Bandit,2017-10-12,0 - 20000,0,0,0.99,0,THE LAW!! Looks to be a showdown atop a train....,"['English', 'French', 'Italian', 'German', 'Sp...",...,0,0,0,0,1,0,1,1,0,0
2,1732930,Jolt Project,2021-11-17,0 - 20000,0,0,4.99,0,Jolt Project: The army now has a new robotics ...,"['English', 'Portuguese - Brazil']",...,0,0,0,0,1,0,0,1,0,0
3,1355720,Henosis™,2020-07-23,0 - 20000,0,0,5.99,0,HENOSIS™ is a mysterious 2D Platform Puzzler w...,"['English', 'French', 'Italian', 'German', 'Sp...",...,0,0,0,0,1,0,1,1,0,0
4,1139950,Two Weeks in Painland,2020-02-03,0 - 20000,0,0,0.00,0,ABOUT THE GAME Play as a hacker who has arrang...,"['English', 'Spanish - Spain']",...,0,0,0,0,1,0,0,1,0,0


In [348]:
print(df_st.columns)

Index(['appid', 'gamename', 'release_date', 'estimated_owners', 'peak_ccu',
       'required_age', 'price', 'dlc_count', 'about_the_game',
       'supported_languages', 'full_audio_languages', 'reviews',
       'header_image', 'website', 'support_url', 'support_email', 'windows',
       'mac', 'linux', 'metacritic_score', 'metacritic_url', 'user_score',
       'positive', 'negative', 'score_rank', 'achievements', 'recommendations',
       'notes', 'average_playtime_forever', 'average_playtime_two_weeks',
       'median_playtime_forever', 'median_playtime_two_weeks', 'developers',
       'publishers', 'categories', 'genres', 'tags', 'screenshots', 'movies',
       'release_year', 'multi_player', 'pvp', 'co-op', 'genre_action',
       'genre_adventure', 'genre_casual', 'genre_sexual_content',
       'genre_strategy', 'genre_sports', 'genre_racing', 'genre_rpg',
       'genre_simulation', 'indie', 'full_audio', 'full_controller_support',
       'age_0_plus', 'age_13_plus', 'age_18_plus'],

In [441]:
values = ['gamename', 'release_date', 
          'required_age', 'price', 'dlc_count', 
          'windows', 'mac', 'linux', 
          'metacritic_score', 'positive', 'negative',
          'developers', 'publishers', 
          'average_playtime_forever', 'average_playtime_two_weeks',
          'median_playtime_forever', 'median_playtime_two_weeks', 
          'multi_player', 'pvp', 'co-op', 
          'genre_action', 'genre_adventure', 'genre_casual', 
          'genre_sexual_content', 'genre_strategy', 'genre_sports', 
          'genre_racing','genre_rpg', 'genre_simulation', 
          'indie', 'full_audio', 'full_controller_support',
          'age_0_plus', 'age_13_plus', 'age_18_plus'
         ]
st = df_st[values]
st.head()

,gamename,release_date,required_age,price,dlc_count,windows,mac,linux,metacritic_score,positive,...,genre_sports,genre_racing,genre_rpg,genre_simulation,indie,full_audio,full_controller_support,age_0_plus,age_13_plus,age_18_plus
0,Galactic Bowling,2008-10-21,0,19.99,0,1,0,0,0,6,...,1,0,0,0,1,0,0,1,0,0
1,Train Bandit,2017-10-12,0,0.99,0,1,1,0,0,53,...,0,0,0,0,1,0,1,1,0,0
2,Jolt Project,2021-11-17,0,4.99,0,1,0,0,0,0,...,0,0,0,0,1,0,0,1,0,0
3,Henosis™,2020-07-23,0,5.99,0,1,1,1,0,3,...,0,0,0,0,1,0,1,1,0,0
4,Two Weeks in Painland,2020-02-03,0,0.00,0,1,1,0,0,50,...,0,0,0,0,1,0,0,1,0,0


In [442]:
df.head()

,gamename,year,month,avg,gain,peak,avg_peak_perc,date
0,Counter-Strike: Global Offensive,2021,February,741013.24,-2196.42,1123485,65.9567%,2021-02-01
1,Counter-Strike: Global Offensive,2021,January,743209.66,25405.91,1124553,66.0893%,2021-01-01
2,Counter-Strike: Global Offensive,2020,December,717803.75,49049.17,1164396,61.646%,2020-12-01
3,Counter-Strike: Global Offensive,2020,November,668754.58,55087.89,1037464,64.4605%,2020-11-01
4,Counter-Strike: Global Offensive,2020,October,613666.69,6816.37,943876,65.0156%,2020-10-01


In [443]:
condition1 = df['gamename'].isin(st['gamename'])
df_extract = df[condition1].set_index('gamename')
# df_extract = df[condition1]
print(df.shape)
print(df_extract.shape)
df_extract.head()

(83789, 8)
(62109, 7)


,year,month,avg,gain,peak,avg_peak_perc,date
gamename,,,,,,,
Counter-Strike: Global Offensive,2021,February,741013.24,-2196.42,1123485,65.9567%,2021-02-01
Counter-Strike: Global Offensive,2021,January,743209.66,25405.91,1124553,66.0893%,2021-01-01
Counter-Strike: Global Offensive,2020,December,717803.75,49049.17,1164396,61.646%,2020-12-01
Counter-Strike: Global Offensive,2020,November,668754.58,55087.89,1037464,64.4605%,2020-11-01
Counter-Strike: Global Offensive,2020,October,613666.69,6816.37,943876,65.0156%,2020-10-01


In [444]:
condition2 = st['gamename'].isin(df['gamename'])
st_extract = st[condition2].set_index('gamename')
# st_extract = st[condition2]
print(st.shape)
print(st_extract.shape)
st_extract.head()

(76985, 35)
(966, 34)


,release_date,required_age,price,dlc_count,windows,mac,linux,metacritic_score,positive,negative,...,genre_sports,genre_racing,genre_rpg,genre_simulation,indie,full_audio,full_controller_support,age_0_plus,age_13_plus,age_18_plus
gamename,,,,,,,,,,,,,,,,,,,,,
Max Payne,2011-01-06,17,3.49,0,1,0,0,89,9516,1114,...,0,0,0,0,0,1,0,0,1,0
Oxygen Not Included,2019-07-30,0,24.99,2,1,1,1,86,82902,3014,...,0,0,0,1,1,1,0,1,0,0
Borderless Gaming,2015-07-23,0,3.99,1,1,0,0,0,2461,336,...,0,0,0,0,0,0,0,1,0,0
Russian Fishing 4,2021-11-10,0,0.00,1,1,0,0,0,19370,5757,...,1,0,1,1,0,0,0,1,0,0
Mafia: Definitive Edition,2020-09-24,17,39.99,1,1,0,0,0,37900,5785,...,0,0,0,0,0,1,1,0,1,0


In [445]:
df_join = df_extract.join(st_extract, how='inner')
df_join = df_join.reset_index()
df_join

,gamename,year,month,avg,gain,peak,avg_peak_perc,date,release_date,required_age,...,genre_sports,genre_racing,genre_rpg,genre_simulation,indie,full_audio,full_controller_support,age_0_plus,age_13_plus,age_18_plus
0,100% Orange Juice,2021,February,278.67,-25.23,645,43.2047%,2021-02-01,2014-05-16,0,...,0,0,0,0,1,1,0,1,0,0
1,100% Orange Juice,2021,January,303.89,-4.16,614,49.4935%,2021-01-01,2014-05-16,0,...,0,0,0,0,1,1,0,1,0,0
2,100% Orange Juice,2020,December,308.05,23.02,743,41.4603%,2020-12-01,2014-05-16,0,...,0,0,0,0,1,1,0,1,0,0
3,100% Orange Juice,2020,November,285.03,-7.41,621,45.8986%,2020-11-01,2014-05-16,0,...,0,0,0,0,1,1,0,1,0,0
4,100% Orange Juice,2020,October,292.45,46.68,900,32.4944%,2020-10-01,2014-05-16,0,...,0,0,0,0,1,1,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
63849,theHunter Classic,2014,October,909.46,258.77,2293,39.6625%,2014-10-01,2014-06-03,0,...,1,0,0,1,0,1,0,1,0,0
63850,theHunter Classic,2014,September,650.69,-63.60,1717,37.8969%,2014-09-01,2014-06-03,0,...,1,0,0,1,0,1,0,1,0,0
63851,theHunter Classic,2014,August,714.29,-337.67,1181,60.4818%,2014-08-01,2014-06-03,0,...,1,0,0,1,0,1,0,1,0,0
63852,theHunter Classic,2014,July,1051.96,-110.53,2059,51.0908%,2014-07-01,2014-06-03,0,...,1,0,0,1,0,1,0,1,0,0


In [447]:
print(df_join.index.unique().value_counts().sum())
print(st_extract.index.value_counts().sum())
df_join.columns

63854
966


Index(['gamename', 'year', 'month', 'avg', 'gain', 'peak', 'avg_peak_perc',
       'date', 'release_date', 'required_age', 'price', 'dlc_count', 'windows',
       'mac', 'linux', 'metacritic_score', 'positive', 'negative',
       'developers', 'publishers', 'average_playtime_forever',
       'average_playtime_two_weeks', 'median_playtime_forever',
       'median_playtime_two_weeks', 'multi_player', 'pvp', 'co-op',
       'genre_action', 'genre_adventure', 'genre_casual',
       'genre_sexual_content', 'genre_strategy', 'genre_sports',
       'genre_racing', 'genre_rpg', 'genre_simulation', 'indie', 'full_audio',
       'full_controller_support', 'age_0_plus', 'age_13_plus', 'age_18_plus'],
      dtype='object')

In [448]:
from pathlib import Path  
filepath = Path('./join_02.csv')  
filepath.parent.mkdir(parents=True, exist_ok=True)
df_join.to_csv(filepath, index=False)

In [354]:
# df_st = df_st.drop(['age-0plus'], axis=1)
# print(df_st['age_18_plus'].value_counts().sum())
# print(df_st.required_age.unique())
# df_st.head()